In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("uitnlp/vietnamese_students_feedback")
dataset = dataset.rename_column("sentence", 'text')
dataset = dataset.rename_column("sentiment", 'label')
dataset = dataset.remove_columns("topic")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset vietnamese_students_feedback downloaded and prepared to /root/.cache/huggingface/datasets/uitnlp___vietnamese_students_feedback/default/1.0.0/b01aa0fea00f1179ef9c093babb5e91f44bc60f19ded600754a08e018503e40f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3166
    })
})

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from transformers import AutoTokenizer

model_name = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [6]:
train_dataset = tokenized_datasets["train"].shuffle(seed=267)
eval_dataset = tokenized_datasets["validation"].shuffle(seed=267)
test_dataset = tokenized_datasets['test'].shuffle(seed=267)

In [8]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

559893507

In [10]:
!pip install -U accelerate
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/f7/fc/c55e5a2da345c9a24aa2e1e0f60eb2ca290b6a41be82da03a6d4baec4f99/accelerate-0.25.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.24.1
    Uninstalling accelerate-0.24.1:
      Successfully uninstalled accelerate-0.24.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
import numpy as np
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': torch.sum(torch.tensor(predictions)==torch.tensor(labels))/len(predictions),
        'f1': f1_score(labels, predictions, average='macro')
    }

In [12]:
import wandb
import os
os.environ['WANDB_PROJECT'] = "experiment"
os.environ['WANDB_LOG_MODEL'] = "true"

In [13]:
import re
run_name=re.sub(r'/', "-", model_name) + "_sentiment_mining_" + "13"

In [14]:
batch_size=2

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer_01_12",
    disable_tqdm=True,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    logging_steps=50,
    report_to="wandb",
    run_name=run_name,
    # save_steps=0.51,
    # lr_scheduler_type="polynomial",
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=5e-6,
    # weight_decay=1e-2,
    do_train=True,
    num_train_epochs=5,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    max_grad_norm=1
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

import torch
torch.cuda.empty_cache()
import wandb
wandb.login(key="e4c568fdefe63f6dcc6f88155c670e1d058abbf8")
output = trainer.train()
output

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huysntq (sfdsafa). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231202_172636-12hav1js
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run xlm-roberta-large_sentiment_mining_13
wandb: ⭐️ View project at https://wandb.ai/sfdsafa/experiment
wandb: 🚀 View run at https://wandb.ai/sfdsafa/experiment/runs/12hav1js


{'loss': 0.911, 'learning_rate': 4.991248030806932e-06, 'epoch': 0.01}
{'loss': 0.8199, 'learning_rate': 4.9824960616138636e-06, 'epoch': 0.02}
{'loss': 0.9339, 'learning_rate': 4.9737440924207956e-06, 'epoch': 0.03}
{'loss': 0.7099, 'learning_rate': 4.964992123227727e-06, 'epoch': 0.04}
{'loss': 0.5791, 'learning_rate': 4.956240154034658e-06, 'epoch': 0.04}
{'loss': 0.4117, 'learning_rate': 4.94748818484159e-06, 'epoch': 0.05}
{'loss': 0.5567, 'learning_rate': 4.938736215648521e-06, 'epoch': 0.06}
{'loss': 0.4613, 'learning_rate': 4.929984246455453e-06, 'epoch': 0.07}
{'loss': 0.6178, 'learning_rate': 4.921232277262384e-06, 'epoch': 0.08}
{'loss': 0.6624, 'learning_rate': 4.912480308069316e-06, 'epoch': 0.09}
{'loss': 0.6491, 'learning_rate': 4.903728338876247e-06, 'epoch': 0.1}
{'loss': 0.5117, 'learning_rate': 4.894976369683179e-06, 'epoch': 0.11}
{'loss': 0.6109, 'learning_rate': 4.8862244004901105e-06, 'epoch': 0.11}
{'loss': 0.7675, 'learning_rate': 4.8774724312970425e-06, 'epoch

TrainOutput(global_step=28565, training_loss=0.27523999629904455, metrics={'train_runtime': 14361.4804, 'train_samples_per_second': 3.978, 'train_steps_per_second': 1.989, 'train_loss': 0.27523999629904455, 'epoch': 5.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.39049938321113586, 'eval_accuracy': 0.953885018825531, 'eval_f1': 0.8710721872231627, 'eval_runtime': 99.054, 'eval_samples_per_second': 15.981, 'eval_steps_per_second': 7.996, 'epoch': 5.0}


{'eval_loss': 0.39049938321113586,
 'eval_accuracy': 0.953885018825531,
 'eval_f1': 0.8710721872231627,
 'eval_runtime': 99.054,
 'eval_samples_per_second': 15.981,
 'eval_steps_per_second': 7.996,
 'epoch': 5.0}